In [22]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os


In [23]:
load_dotenv() 
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

In [24]:
loader = PyPDFLoader("documents/ctse.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

In [25]:
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding)
retriever = vectorstore.as_retriever()

In [26]:
prompt_template = ChatPromptTemplate.from_template("""
You are an intelligent assistant helping software engineering students understand their lecture material.

Use the following retrieved lecture content to answer the user's question clearly, concisely, and helpfully. If the context is not sufficient to answer the question, say so honestly.

Retrieved Lecture Context:
{context}

Student Question:
{question}

Answer:
""")

In [27]:
llm = OpenAI(temperature=0.7)
document_chain = create_stuff_documents_chain(llm, prompt_template)


In [28]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)

In [ ]:

while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("Goodbye!")
        break
    

    response = rag_chain({"query": user_query})


    

    if isinstance(response, dict):

        result = response.get("result", response.get("answer", response.get("output", str(response))))
        print(f"Bot: {result}\n")
    else:
        print(f"Bot: {response}\n")

Bot: 
DevOps is a set of practices that aim to improve collaboration and communication between software development and operations teams. It promotes a culture of shared responsibility and ownership among team members, reduces organizational silos, and encourages everyone to work towards a common goal of delivering high-quality software efficiently. By implementing DevOps, teams can measure everything from application performance to system monitoring and metrics, allowing for continuous improvement and faster response to issues. Gradual changes and frequent deployments also help in ensuring a stable and reliable software release process. Additionally, DevOps encourages a mindset of accepting failure as normal and focuses on blameless post-mortems and risk-taking to drive innovation. Leveraging tooling and automation is also a key aspect of DevOps, as it can help reduce manual work and improve efficiency. Overall, DevOps helps teams deliver better software faster and with fewer errors, 